## Nume studenti:
- Alexandra Manole
- Teodor Mihaescu

## Grupa: 382

# Data Set 2: Energy Efficency
### (Missing values: no)

In [1171]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV

In [1155]:
# Citeste datele
data = pd.read_csv('./Datasets/energy.csv').iloc[:768,:-2]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    float64
 6   X7      768 non-null    float64
 7   X8      768 non-null    float64
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(10)
memory usage: 60.1 KB


In [1156]:
# Pregateste datele
X = data.loc[:, :'X8']
# y = data.loc[:, 'Y1':]
y1 = data.loc[:, 'Y1']
y2 = data.loc[:, 'Y2']

In [1157]:
# Transformam problema dintr-una de regresie intr-una de clasificare, dand intervale de clasificare y-ilor
"""
Observam ca y1 si y2 se incadreaza intre urmatoarele valori:
y1: 6.01-43.1
y2: 10.9-48.03

Vom imparti datele in urmatoarele clase:
y1: 6-10, 10-20, 20-30, 30-40, 40-50
y2: 10-20, 20-30, 30-40, 40-50
"""
y1 = np.where(np.logical_and(y1>=6, y1<10), 0, y1)
y1 = np.where(np.logical_and(y1>=10, y1<20), 1, y1)
y1 = np.where(np.logical_and(y1>=20, y1<30), 2, y1)
y1 = np.where(np.logical_and(y1>=30, y1<40), 3, y1)
y1 = np.where(np.logical_and(y1>=40, y1<50), 4, y1)

y2 = np.where(np.logical_and(y2>=10, y2<20), 0, y2)
y2 = np.where(np.logical_and(y2>=20, y2<30), 1, y2)
y2 = np.where(np.logical_and(y2>=30, y2<40), 2, y2)
y2 = np.where(np.logical_and(y2>=40, y2<50), 3, y2)

y = np.stack((y1, y2), axis=1)

In [1158]:
# Scaleaza datele
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
# y = min_max_scaler.fit_transform(y)

In [1159]:
# Separarea setului de date
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True)

In [1160]:
# Verifica daca sunt egal distribuite
print(Counter(y_train[:,0]))
print(Counter(y_test[:,0]))
print("\n")
print(Counter(y_train[:,1]))
print(Counter(y_test[:,1]))

Counter({1.0: 262, 3.0: 110, 2.0: 102, 4.0: 24, 0.0: 14})
Counter({1.0: 115, 2.0: 67, 3.0: 57, 4.0: 11, 0.0: 6})


Counter({0.0: 243, 2.0: 135, 1.0: 103, 3.0: 31})
Counter({0.0: 105, 2.0: 81, 1.0: 62, 3.0: 8})


## Model 1: LogisticRegression

In [1161]:
# Model
"""
"""
model1 = MultiOutputClassifier(LogisticRegression(random_state=0, multi_class='multinomial'))
model1.fit(X_train, y_train)
y_hat = model1.predict(X_test)

In [1162]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[3. 2.]
 [2. 1.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]]

Ground truth:
[[2. 2.]
 [2. 1.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [4. 2.]]

Failed:	[50 54]


In [1163]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [1164]:
# Custom scoreers for multiclass output
f1_scorer = metrics.make_scorer(metrics.f1_score, greater_is_better=False, average='micro')
accuracy_scorer = metrics.make_scorer(metrics.accuracy_score, greater_is_better=True)

In [1165]:
# CV Train - Y1
results_train = cross_validate(model1, X_train, y1_train,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.03690171, 0.03691125, 0.03391552, 0.0398922 , 0.04288459]),
 'score_time': array([0.00299191, 0.00498104, 0.00199223, 0.00299287, 0.00099754]),
 'test_f1_scorer': array([-0.81553398, -0.87378641, -0.81372549, -0.85294118, -0.84313725]),
 'train_f1_scorer': array([-0.84352078, -0.83129584, -0.85121951, -0.84390244, -0.83902439]),
 'test_accuracy': array([0.81553398, 0.87378641, 0.81372549, 0.85294118, 0.84313725]),
 'train_accuracy': array([0.84352078, 0.83129584, 0.85121951, 0.84390244, 0.83902439])}

In [1166]:
# CV Test - Y1
results_train = cross_validate(model1, X_test, y1_test,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.04287887, 0.03989077, 0.03184557, 0.02493596, 0.02593279]),
 'score_time': array([0.0019958 , 0.00199604, 0.00202632, 0.00199413, 0.0030303 ]),
 'test_f1_scorer': array([-0.86538462, -0.80392157, -0.80392157, -0.82352941, -0.7254902 ]),
 'train_f1_scorer': array([-0.81372549, -0.83414634, -0.83414634, -0.8195122 , -0.84878049]),
 'test_accuracy': array([0.86538462, 0.80392157, 0.80392157, 0.82352941, 0.7254902 ]),
 'train_accuracy': array([0.81372549, 0.83414634, 0.83414634, 0.8195122 , 0.84878049])}

In [1167]:
# CV Train - Y2
results_train = cross_validate(model1, X_train, y2_train,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.04987097, 0.04881907, 0.04471707, 0.0442729 , 0.04536247]),
 'score_time': array([0.00399017, 0.00499606, 0.0019803 , 0.00195432, 0.00200009]),
 'test_f1_scorer': array([-0.81553398, -0.74757282, -0.75490196, -0.76470588, -0.79411765]),
 'train_f1_scorer': array([-0.76528117, -0.7799511 , -0.77073171, -0.78292683, -0.76829268]),
 'test_accuracy': array([0.81553398, 0.74757282, 0.75490196, 0.76470588, 0.79411765]),
 'train_accuracy': array([0.76528117, 0.7799511 , 0.77073171, 0.78292683, 0.76829268])}

In [1168]:
# CV Test - Y2
results_train = cross_validate(model1, X_test, y2_test,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.04189301, 0.03291178, 0.01994944, 0.01956725, 0.01997685]),
 'score_time': array([0.00398445, 0.00199318, 0.00299287, 0.00199389, 0.00196147]),
 'test_f1_scorer': array([-0.73076923, -0.82352941, -0.80392157, -0.76470588, -0.84313725]),
 'train_f1_scorer': array([-0.83823529, -0.80487805, -0.81463415, -0.80487805, -0.8097561 ]),
 'test_accuracy': array([0.73076923, 0.82352941, 0.80392157, 0.76470588, 0.84313725]),
 'train_accuracy': array([0.83823529, 0.80487805, 0.81463415, 0.80487805, 0.8097561 ])}

In [1169]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {
    'C' : np.linspace(1000, 2000, 10),# C = 1/lambda ; C mare => lambda mic
    'solver': ['saga', 'lbfgs']}
grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [1170]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.8473604826546003
{'C': 1111.111111111111, 'solver': 'lbfgs'}


## Model 2: KNeighborsClassifier

In [1174]:
# Model
"""
"""
model2 = KNeighborsClassifier()
model2.fit(X_train, y_train)
y_hat = model2.predict(X_test)

In [1175]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[3. 2.]
 [3. 1.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 1.]
 [1. 0.]
 [2. 1.]]

Ground truth:
[[2. 2.]
 [2. 1.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [4. 2.]]

Failed:	[50 44]


In [1176]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [1177]:
# CV Train - Y1
results_train = cross_validate(model2, X_train, y1_train,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.04089355, 0.02891946, 0.03390813, 0.02895498, 0.04185629]),
 'score_time': array([0.00199294, 0.00299263, 0.0009973 , 0.00099993, 0.00298786]),
 'test_f1_scorer': array([-0.81553398, -0.87378641, -0.81372549, -0.85294118, -0.84313725]),
 'train_f1_scorer': array([-0.84352078, -0.83129584, -0.85121951, -0.84390244, -0.83902439]),
 'test_accuracy': array([0.81553398, 0.87378641, 0.81372549, 0.85294118, 0.84313725]),
 'train_accuracy': array([0.84352078, 0.83129584, 0.85121951, 0.84390244, 0.83902439])}

In [1178]:
# CV Test - Y1
results_train = cross_validate(model2, X_test, y1_test,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.0349102 , 0.01794362, 0.02198577, 0.02092791, 0.01595592]),
 'score_time': array([0.0039916 , 0.0020287 , 0.00298905, 0.00099945, 0.00199628]),
 'test_f1_scorer': array([-0.86538462, -0.80392157, -0.80392157, -0.82352941, -0.7254902 ]),
 'train_f1_scorer': array([-0.81372549, -0.83414634, -0.83414634, -0.8195122 , -0.84878049]),
 'test_accuracy': array([0.86538462, 0.80392157, 0.80392157, 0.82352941, 0.7254902 ]),
 'train_accuracy': array([0.81372549, 0.83414634, 0.83414634, 0.8195122 , 0.84878049])}

In [1179]:
# CV Train - Y2
results_train = cross_validate(model2, X_train, y2_train,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.04587817, 0.03391075, 0.034899  , 0.0249331 , 0.03690052]),
 'score_time': array([0.00099897, 0.00199485, 0.0020256 , 0.00296164, 0.00199485]),
 'test_f1_scorer': array([-0.81553398, -0.74757282, -0.75490196, -0.76470588, -0.79411765]),
 'train_f1_scorer': array([-0.76528117, -0.7799511 , -0.77073171, -0.78292683, -0.76829268]),
 'test_accuracy': array([0.81553398, 0.74757282, 0.75490196, 0.76470588, 0.79411765]),
 'train_accuracy': array([0.76528117, 0.7799511 , 0.77073171, 0.78292683, 0.76829268])}

In [1180]:
# CV Test - Y2
results_train = cross_validate(model2, X_test, y2_test,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.03691149, 0.01894999, 0.02294016, 0.01791954, 0.01599216]),
 'score_time': array([0.00198936, 0.00295949, 0.00202513, 0.00199318, 0.00199008]),
 'test_f1_scorer': array([-0.73076923, -0.82352941, -0.80392157, -0.76470588, -0.84313725]),
 'train_f1_scorer': array([-0.83823529, -0.80487805, -0.81463415, -0.80487805, -0.8097561 ]),
 'test_accuracy': array([0.73076923, 0.82352941, 0.80392157, 0.76470588, 0.84313725]),
 'train_accuracy': array([0.83823529, 0.80487805, 0.81463415, 0.80487805, 0.8097561 ])}

In [1184]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'n_neighbors': list(range(1, 20)), 'p': [1, 2, 3]}
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [1185]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.8515837104072398
{'n_neighbors': 3, 'p': 1}


## Model 3: DecisionTreeClassifier

In [1186]:
# Model
"""
"""
model3 = DecisionTreeClassifier(max_depth=2)
model3.fit(X_train, y_train)
y_hat = model3.predict(X_test)

In [1187]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[2. 2.]
 [2. 2.]
 [1. 0.]
 [2. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]]

Ground truth:
[[2. 2.]
 [2. 1.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [4. 2.]]

Failed:	[53 70]


In [1188]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [1190]:
# CV Train - Y1
results_train = cross_validate(model3, X_train, y1_train,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099778, 0.00099707, 0.0019865 , 0.00099254, 0.00099421]),
 'score_time': array([0.00199461, 0.00299311, 0.00598145, 0.0019927 , 0.00099897]),
 'test_f1_scorer': array([-0.7961165 , -0.83495146, -0.84313725, -0.81372549, -0.84313725]),
 'train_f1_scorer': array([-0.83374083, -0.82396088, -0.82195122, -0.82926829, -0.82195122]),
 'test_accuracy': array([0.7961165 , 0.83495146, 0.84313725, 0.81372549, 0.84313725]),
 'train_accuracy': array([0.83374083, 0.82396088, 0.82195122, 0.82926829, 0.82195122])}

In [1194]:
# CV Test - Y1
results_train = cross_validate(model3, X_test, y1_test,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00199866, 0.00099897, 0.0009973 , 0.0019989 , 0.0009532 ]),
 'score_time': array([0.00299358, 0.00299072, 0.00199509, 0.00202799, 0.00303531]),
 'test_f1_scorer': array([-0.78846154, -0.76470588, -0.84313725, -0.84313725, -0.7254902 ]),
 'train_f1_scorer': array([-0.79411765, -0.8       , -0.7804878 , -0.7804878 , -0.8097561 ]),
 'test_accuracy': array([0.78846154, 0.76470588, 0.84313725, 0.84313725, 0.7254902 ]),
 'train_accuracy': array([0.79411765, 0.8       , 0.7804878 , 0.7804878 , 0.8097561 ])}

In [1191]:
# CV Train - Y2
results_train = cross_validate(model3, X_train, y2_train,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.0009644 , 0.00199175, 0.00099611, 0.00098538, 0.00100994]),
 'score_time': array([0.00199723, 0.00199533, 0.00299191, 0.00399017, 0.00297356]),
 'test_f1_scorer': array([-0.73786408, -0.73786408, -0.74509804, -0.73529412, -0.73529412]),
 'train_f1_scorer': array([-0.73838631, -0.73838631, -0.73658537, -0.73902439, -0.73902439]),
 'test_accuracy': array([0.73786408, 0.73786408, 0.74509804, 0.73529412, 0.73529412]),
 'train_accuracy': array([0.73838631, 0.73838631, 0.73658537, 0.73902439, 0.73902439])}

In [1192]:
# CV Test - Y2
results_train = cross_validate(model3, X_test, y2_test,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.        , 0.        , 0.00099635, 0.00099635, 0.        ]),
 'score_time': array([0.00152397, 0.00199199, 0.00398684, 0.00199628, 0.00099683]),
 'test_f1_scorer': array([-0.69230769, -0.74509804, -0.76470588, -0.68627451, -0.78431373]),
 'train_f1_scorer': array([-0.75490196, -0.75121951, -0.74634146, -0.76585366, -0.74146341]),
 'test_accuracy': array([0.69230769, 0.74509804, 0.76470588, 0.68627451, 0.78431373]),
 'train_accuracy': array([0.75490196, 0.75121951, 0.74634146, 0.76585366, 0.74146341])}

In [1199]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'max_depth': list(range(1,5)),
'min_samples_split': np.linspace(1,3,5),
'min_samples_leaf': np.linspace(0.1,0.5,5),
'max_features': ['sqrt', 'log2', None],
'criterion': ['gini', 'entropy']}
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [1200]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.6296380090497739
{'criterion': 'gini', 'max_depth': 1, 'max_features': 'log2', 'min_samples_leaf': 0.30000000000000004, 'min_samples_split': 1.0}


## Model 4: RandomForestClassifier

In [ ]:
# Model
"""
"""
model4 = RandomForestClassifier(max_depth=2, random_state=0, ccp_alpha=0.2)
model4.fit(X_train, y_train)
y_hat = model4.predict(X_test)